In [1]:
# Функция для создания датасета для LSTM
def create_lstm_dataset(data, time_steps=1, forecast_days=1):
    X, y = [], []
    for i in range(len(data) - time_steps - forecast_days + 1):
        end_ix = i + time_steps
        out_end_ix = end_ix + forecast_days
        if out_end_ix > len(data):
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

from free_utils import (
    file_names,
    n_intervals
)

In [3]:
file_id = 4
file_path = '../data_updated/' + file_names[file_id]

# Read the file into a DataFrame
dataset = pd.read_csv(file_path)
dataset

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,SMA_20,EMA_20,RSI,MACD,Signal_Line
0,2013-01-14,250.50,249.19,251.69,246.37,12173000,1.09,0,1,1,2013,14,14,3,254.5075,253.922471,34.080189,-4.536622,-4.399342
1,2013-01-15,251.00,251.50,255.00,247.33,15977000,0.20,1,1,1,2013,15,15,3,255.2090,254.282732,34.486874,-4.853714,-4.365022
2,2013-01-16,252.50,251.01,254.98,248.60,9309000,0.60,2,1,1,2013,16,16,3,256.1885,254.628282,35.925926,-5.238748,-4.242849
3,2013-01-17,245.49,251.70,253.47,245.00,21241000,-2.78,3,1,1,2013,17,17,3,257.2885,254.852312,22.447157,-5.806719,-3.993875
4,2013-01-18,245.82,246.00,246.89,239.01,19380000,0.13,4,1,1,2013,18,18,3,258.8640,255.837818,22.222222,-5.733450,-3.540664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,2023-10-09,189.10,187.80,191.90,187.70,57131000,0.96,0,4,10,2023,282,9,41,NaN,190.690216,NaN,-0.164748,-0.046064
2688,2023-10-10,191.70,190.00,192.00,189.10,19047000,1.37,1,4,10,2023,283,10,41,NaN,190.857607,NaN,-0.028207,-0.016393
2689,2023-10-11,188.80,192.00,193.60,187.60,54770000,-1.51,2,4,10,2023,284,11,41,NaN,190.768934,NaN,-0.118249,-0.013439
2690,2023-10-12,191.70,188.00,193.00,187.20,41655000,1.54,3,4,10,2023,285,12,41,NaN,190.976190,NaN,0.063818,0.012764


In [4]:
    
df = dataset
df = df.dropna()
# Извлекаем целевую переменную и столбец с датами
target_column = df.pop('Цена')
dates = df.pop('Дата')

# Вставляем целевую переменную в конец DataFrame
df['Цена'] = target_column

# Преобразуем в numpy массив
dataset = df.values


C:\Users\User\AppData\Local\Temp\ipykernel_17288\3912821207.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Цена'] = target_column


In [5]:
    
# Создаем датасет для LSTM
time_steps = n_intervals  # Количество временных шагов
forecast_days = 1  # Количество дней для предсказания
# Normalize the dataset using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_dataset = scaler.fit_transform(dataset)

X_ltsm, y_ltsm = create_lstm_dataset(normalized_dataset, time_steps, forecast_days)


In [6]:

# Разделяем на обучающий и тестовый наборы данных
train_size = int(len(X_ltsm) * 0.8)
X_train_ltsm, X_test_ltsm = X_ltsm[:train_size], X_ltsm[train_size:]
y_train_ltsm, y_testt_ltsm = y_ltsm[:train_size], y_ltsm[train_size:]
# Сохраняем scaler_X и scaler_y в файлы

scaler_filename = '../scalers/'+file_names[file_id]+'_scaler.pkl'
with open(scaler_filename, 'wb') as scaler_file:
    joblib.dump(scaler, scaler_file)


# Выводим размеры полученных датасетов
print("Размеры X_train_ltsm, y_train_ltsm, X_test_ltsm, y_test_ltsm:")
print(X_train_ltsm.shape, y_train_ltsm.shape, X_test_ltsm.shape, y_testt_ltsm.shape)


Размеры X_train_ltsm, y_train_ltsm, X_test_ltsm, y_test_ltsm:
(2042, 120, 18) (2042, 1) (511, 120, 18) (511, 1)


In [7]:
    import os
    
    # Путь к папке, где будем сохранять датасеты
    datasets_folder = '../datasets'
    
    # Создадим папку, если её нет
    os.makedirs(datasets_folder, exist_ok=True)
    
    # Сохранение данных X_train и y_train_ltsm
    X_train_ltsm_file_path = os.path.join(datasets_folder, f'X_train_ltsm_{file_names[file_id]}')
    X_test_ltsm_file_path = os.path.join(datasets_folder, f'X_test_ltsm_{file_names[file_id]}')
    y_train_ltsm_file_path = os.path.join(datasets_folder, f'y_train_ltsm_{file_names[file_id]}')
    y_test_ltsm_file_path = os.path.join(datasets_folder, f'y_test_ltsm_{file_names[file_id]}')
    
    np.save(X_train_ltsm_file_path, X_train_ltsm)
    np.save(X_test_ltsm_file_path, X_test_ltsm)
    
    np.save(y_train_ltsm_file_path, y_train_ltsm)
    np.save(y_test_ltsm_file_path, y_testt_ltsm)
    
    print(f'Data saved to {X_train_ltsm_file_path} and {X_test_ltsm_file_path}')
    print(f'Data saved to {y_train_ltsm_file_path} and {y_test_ltsm_file_path}')


Data saved to ../datasets\X_train_ltsm_Прошлые данные - MVID.csv and ../datasets\X_test_ltsm_Прошлые данные - MVID.csv
Data saved to ../datasets\y_train_ltsm_Прошлые данные - MVID.csv and ../datasets\y_test_ltsm_Прошлые данные - MVID.csv


In [8]:
# dataset_with_shift=dataset
# #Сдвигаем на одну позицию вперед целевую переменную
# #Target_1 это лаг наооборот для целевой переменной , типа предсказываем на день вперед
# # dataset_with_shift['future_1'] = dataset['Цена'].shift(1)
# # dataset_with_shift.dropna(inplace=True)
# # 
# # dataset_with_shift
# 
# X_xgboost = dataset.drop(['Цена','Дата'],axis=1)
# y_xgboost = dataset['Target_1']  # Целевая переменная
# X_xgboost = y_xgboost[-len(X_ltsm):]
# y_xgboost = X_xgboost[-len(X_ltsm):]